In [1]:
%matplotlib inline

import dateutil.parser
import datetime
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.contrib import learn

from pymongo import MongoClient
from bson.objectid import ObjectId

from sklearn.metrics import mean_squared_error

from lstm import lstm_model
from data_processing import generate_data, load_csvdata

In [2]:
LOG_DIR = './ops_logs/data_stock1'
TIMESTEPS = 1
RNN_LAYERS = [{'num_unitss':5}]
DENSE_LAYERS = [10, 10]
TRAINING_STEPS = 100000
BATCH_SIZE = 1
PRINT_STEPS = TRAINING_STEPS /100

In [3]:
def load_stock_frame(filename):
    data_raw = pd.read_csv(filename, dtype={'TIME': str})
    data_raw['MEANALL'] = data_raw['MEANALL'].astype(float)
    times = []
    for index, row in data_raw.iterrows():
        _t = datetime.datetime.strptime(row['TIME'], "%y%m%d")
        times.append(_t)
        
    data_raw['_time'] = pd.Series(times, index=data_raw.index)
    df = pd.DataFrame(data_raw, columns=['TIME', 'MEANALL'])
    return df.set_index('TIME')

In [4]:
data_stock = load_stock_frame("mydata/dailydata4.csv")
X, y = load_csvdata(data_stock, TIMESTEPS, seperate=False)

In [5]:
regressor = learn.SKCompat(learn.Estimator(
        model_fn=lstm_model(
            TIMESTEPS,
            RNN_LAYERS,
            DENSE_LAYERS
        ),
        model_dir=LOG_DIR
))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f413585bd50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [6]:
validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],
                                                     every_n_steps=PRINT_STEPS,
                                                     early_stopping_rounds=1000)
regressor.fit(X['train'], y['train'],
             monitors=[validation_monitor],
             batch_size=BATCH_SIZE,
             steps=TRAINING_STEPS)

predicted = regressor.predict(X['test'])

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


KeyError: 'num_units'